# LAB 2 – Tools y CRUD de tickets con `IntakeAgent`

Partimos del escenario del **Service Desk inteligente** y del agente `IntakeAgentJSON` (LAB 1), que devuelve un JSON con la clasificación de la solicitud.

En este LAB 2 vamos a:

- Cargar configuración de **GitHub Models** desde un fichero `.env` (endpoint, token, modelo).
- Reutilizar un agente `IntakeAgentJSON` que devuelve JSON.
- Implementar herramientas (functions) en Python para:
  - Crear tickets en `data/tickets.csv`.
  - Listar y buscar tickets por email y texto.
  - Guardar tickets como JSON en `out/`.
- Componer un flujo: **texto libre → JSON (agente) → ticket guardado**.


## 1. Configuración mediante `.env`

Crea un fichero `.env` en la raíz del proyecto (`service_desk/`) con:

```env
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_TOKEN=ghp_xxx_tu_token_xxx
GITHUB_MODEL=gpt-4o


In [1]:

import os
from dotenv import load_dotenv

# Cargar variables desde .env
load_dotenv()

GITHUB_ENDPOINT = os.getenv("GITHUB_ENDPOINT")
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
GITHUB_MODEL = os.getenv("GITHUB_MODEL")

if not all([GITHUB_ENDPOINT, GITHUB_TOKEN, GITHUB_MODEL]):
    raise RuntimeError(
        "❌ Faltan variables en .env. Asegúrate de tener "
        "GITHUB_MODELS_ENDPOINT, GITHUB_MODELS_TOKEN y GITHUB_MODELS_MODEL configuradas."
    )

# Adaptar a las variables esperadas por OpenAIChatClient (Agent Framework)
os.environ["OPENAI_API_KEY"] = GITHUB_TOKEN
os.environ["OPENAI_BASE_URL"] = GITHUB_ENDPOINT
os.environ["OPENAI_CHAT_MODEL_ID"] = GITHUB_MODEL

print("✅ Configuración cargada desde .env")
print("  ENDPOINT:", GITHUB_ENDPOINT)
print("  MODEL:", GITHUB_MODEL)


✅ Configuración cargada desde .env
  ENDPOINT: https://models.github.ai/inference
  MODEL: gpt-4o


In [2]:
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient

import asyncio
import json
from pathlib import Path
import pandas as pd

# Directorios base
DATA_DIR = Path("data")
OUT_DIR = Path("out")
DATA_DIR.mkdir(exist_ok=True)
OUT_DIR.mkdir(exist_ok=True)

print("📂 DATA_DIR:", DATA_DIR.resolve())
print("📂 OUT_DIR:", OUT_DIR.resolve())

# Instrucciones del agente intake (versión resumida del LAB 1)
INTAKE_JSON_INSTRUCTIONS = """
Eres un agente de primer nivel de un Service Desk interno.

Tu tarea es LEER la solicitud del usuario y devolver SIEMPRE un único objeto JSON con esta estructura:

{
  "departamento": "IT | RRHH | Facilities | Otro",
  "categoria": "nuevo_equipo | incidencia | vacaciones | certificado | mantenimiento | otro",
  "prioridad": "alta | media | baja",
  "resumen": "Texto corto que resuma la solicitud",
  "detalle": "Texto con más contexto, si es necesario"
}

Reglas IMPORTANTES:
- No expliques nada fuera del JSON.
- No añadas comentarios ni texto antes o después del JSON.
- Rellena los campos en base a la intención del usuario.
- Si tienes dudas, elige el valor más razonable y utiliza "otro" en categoria/departamento cuando no encaje.
- La prioridad será:
  - "alta" si hay urgencia, bloqueo, caída de servicio o impacto fuerte.
  - "media" en la mayoría de solicitudes estándar.
  - "baja" para dudas generales o temas no urgentes.
"""

async def create_intake_agent_json() -> ChatAgent:
    """Crea una instancia de IntakeAgentJSON usando el modelo configurado en .env."""
    # OpenAIChatClient lee OPENAI_API_KEY, OPENAI_BASE_URL, OPENAI_CHAT_MODEL_ID
    chat_client = OpenAIChatClient()
    agent = chat_client.create_agent(
        name="IntakeAgentJSON",
        instructions=INTAKE_JSON_INSTRUCTIONS,
    )
    return agent

async def demo_intake_agent_json_simple():
    """Comprobar que el agente responde JSON correcto."""
    agent = await create_intake_agent_json()
    thread = agent.get_new_thread()

    consulta = "Necesito un portátil nuevo con 16GB de RAM para teletrabajar 3 días a la semana."
    print(f"👤 Usuario: {consulta}\n")

    result = await agent.run(consulta, thread=thread)
    print("🤖 IntakeAgentJSON (salida bruta):\n")
    print(result.text)

await demo_intake_agent_json_simple()


📂 DATA_DIR: C:\Repos\maf-workshop\data
📂 OUT_DIR: C:\Repos\maf-workshop\out
👤 Usuario: Necesito un portátil nuevo con 16GB de RAM para teletrabajar 3 días a la semana.

🤖 IntakeAgentJSON (salida bruta):

{
  "departamento": "IT",
  "categoria": "nuevo_equipo",
  "prioridad": "media",
  "resumen": "Solicitud de portátil con 16GB de RAM para teletrabajo",
  "detalle": "El usuario necesita un portátil con 16GB de RAM para teletrabajar 3 días a la semana."
}


In [3]:
# Helpers para trabajar con data/tickets.csv

TICKETS_CSV = DATA_DIR / "tickets.csv"

# Si el fichero no existe, lo inicializamos con cabecera
if not TICKETS_CSV.exists():
    df_init = pd.DataFrame(
        columns=[
            "id", "fecha", "solicitante", "departamento",
            "categoria", "prioridad", "estado", "resumen", "detalle"
        ]
    )
    df_init.to_csv(TICKETS_CSV, index=False, encoding="utf-8")
    print(f"✅ Fichero de tickets creado: {TICKETS_CSV}")
else:
    print(f"✅ Fichero de tickets encontrado: {TICKETS_CSV}")

def load_tickets() -> pd.DataFrame:
    """Carga el CSV de tickets en un DataFrame de pandas."""
    if TICKETS_CSV.exists():
        return pd.read_csv(TICKETS_CSV, dtype=str).fillna("")
    return pd.DataFrame(
        columns=[
            "id", "fecha", "solicitante", "departamento",
            "categoria", "prioridad", "estado", "resumen", "detalle"
        ]
    )

df_preview = load_tickets()
print("🔎 Vista previa de tickets.csv:")
display(df_preview.head())


✅ Fichero de tickets encontrado: data\tickets.csv
🔎 Vista previa de tickets.csv:


,id,fecha,solicitante,departamento,categoria,prioridad,estado,resumen,detalle
0,1,2025-01-10,antonio.soto@empresa.local,IT,nuevo_equipo,alta,pendiente,Solicitud portátil para teletrabajo,Necesito un portátil con al menos 16GB de RAM ...
1,2,2025-01-12,maria.fraga@empresa.local,RRHH,certificado,media,resuelto_auto,Certificado de empresa,Solicito un certificado de empresa para presen...
2,3,2025-01-15,carlos.gomez@empresa.local,IT,incidencia,alta,en_progreso,VPN no funciona,No puedo conectar a la VPN corporativa desde m...
3,4,2025-01-18,laura.fernandez@empresa.local,Facilities,mantenimiento,media,pendiente,Problema con aire acondicionado,En la sala de reuniones 3A el aire acondiciona...
4,5,2025-01-20,antonio.soto@empresa.local,RRHH,vacaciones,baja,resuelto_auto,Consulta sobre días de vacaciones,Quisiera confirmar cuántos días de vacaciones ...


In [4]:
from datetime import datetime
from typing import List, Dict

def crear_ticket(desde_email: str, ticket_json_str: str) -> Dict:
    """
    Tool: crear un nuevo ticket en tickets.csv a partir del JSON devuelto por IntakeAgent.

    - desde_email: email del solicitante.
    - ticket_json_str: cadena JSON con los campos: departamento, categoria, prioridad, resumen, detalle.
    """
    data = load_tickets()

    # Calcular siguiente ID
    if data.empty:
        next_id = 1
    else:
        ids = pd.to_numeric(data["id"], errors="coerce")
        next_id = int(ids.max() or 0) + 1

    # Parsear JSON del agente
    try:
        ticket_data = json.loads(ticket_json_str)
    except json.JSONDecodeError as e:
        raise ValueError(f"No se pudo parsear el JSON del ticket: {e}")

    nuevo_ticket = {
        "id": str(next_id),
        "fecha": datetime.now().strftime("%Y-%m-%d"),
        "solicitante": desde_email,
        "departamento": ticket_data.get("departamento", "Otro"),
        "categoria": ticket_data.get("categoria", "otro"),
        "prioridad": ticket_data.get("prioridad", "media"),
        "estado": "pendiente",
        "resumen": ticket_data.get("resumen", "")[:200],
        "detalle": ticket_data.get("detalle", ""),
    }

    # Añadir al DataFrame y guardar
    data = pd.concat([data, pd.DataFrame([nuevo_ticket])], ignore_index=True)
    data.to_csv(TICKETS_CSV, index=False, encoding="utf-8")

    # Guardar también un JSON individual del ticket en out/
    ticket_json_path = OUT_DIR / f"ticket_{next_id}.json"
    with ticket_json_path.open("w", encoding="utf-8") as f:
        json.dump(nuevo_ticket, f, ensure_ascii=False, indent=2)

    print(f"✅ Ticket creado con id={next_id}")
    print(f"   - CSV: {TICKETS_CSV}")
    print(f"   - JSON individual: {ticket_json_path}")

    return nuevo_ticket


def listar_tickets_por_email(email: str) -> List[Dict]:
    """Tool: devuelve una lista de tickets asociados a un email dado."""
    data = load_tickets()
    if data.empty:
        return []
    filtrado = data[data["solicitante"].str.lower() == email.lower()]
    return filtrado.to_dict(orient="records")

def load_tickets() -> pd.DataFrame:
    """
    Carga el CSV de tickets en un DataFrame de pandas.
    Si no existe o está vacío, devuelve un DataFrame con las columnas estándar.
    """
    if TICKETS_CSV.exists():
        return pd.read_csv(TICKETS_CSV, dtype=str).fillna("")
    return pd.DataFrame(
        columns=[
            "id", "fecha", "solicitante", "departamento",
            "categoria", "prioridad", "estado", "resumen", "detalle"
        ]
    )

def buscar_tickets_por_texto(email: str, texto: str) -> List[Dict]:
    """Tool: busca tickets del usuario cuyo resumen o detalle contenga el texto dado."""
    data = load_tickets()
    if data.empty:
        return []

    mask_email = data["solicitante"].str.lower() == email.lower()
    mask_text = data["resumen"].str.contains(texto, case=False, na=False) | \
                data["detalle"].str.contains(texto, case=False, na=False)

    filtrado = data[mask_email & mask_text]
    return filtrado.to_dict(orient="records")


In [5]:
# Prueba rápida de las tools sin pasar todavía por el agente Intake

ejemplo_ticket_json = json.dumps({
    "departamento": "IT",
    "categoria": "nuevo_equipo",
    "prioridad": "alta",
    "resumen": "Solicitud de portátil para teletrabajo",
    "detalle": "Necesito un portátil con 16GB de RAM para trabajar desde casa tres días a la semana."
}, ensure_ascii=False)

nuevo = crear_ticket("antonio.soto@empresa.local", ejemplo_ticket_json)
print("Nuevo ticket creado:", nuevo)

print("\nListado de tickets de antonio.soto@empresa.local")
for t in listar_tickets_por_email("antonio.soto@empresa.local"):
    print("-", t["id"], t["resumen"])

print("\nBúsqueda por texto 'portátil' para antonio.soto@empresa.local")
for t in buscar_tickets_por_texto("antonio.soto@empresa.local", "portátil"):
    print("-", t["id"], t["resumen"])


✅ Ticket creado con id=14
   - CSV: data\tickets.csv
   - JSON individual: out\ticket_14.json
Nuevo ticket creado: {'id': '14', 'fecha': '2025-11-28', 'solicitante': 'antonio.soto@empresa.local', 'departamento': 'IT', 'categoria': 'nuevo_equipo', 'prioridad': 'alta', 'estado': 'pendiente', 'resumen': 'Solicitud de portátil para teletrabajo', 'detalle': 'Necesito un portátil con 16GB de RAM para trabajar desde casa tres días a la semana.'}

Listado de tickets de antonio.soto@empresa.local
- 1 Solicitud portátil para teletrabajo
- 5 Consulta sobre días de vacaciones
- 6 Solicitud de portátil para teletrabajo
- 7 Solicitud de portátil nuevo con 16GB de RAM
- 8 Solicitud de portátil con 16GB de RAM
- 9 Solicitud de portátil con 16GB de RAM para teletrabajo
- 10 Solicitud de portátil nuevo con 16GB de RAM
- 11 Solicitud de portátil para teletrabajo
- 12 Solicitud de portátil nuevo con 16GB de RAM
- 13 Solicitud de portátil nuevo con 16GB de RAM.
- 14 Solicitud de portátil para teletrabajo



In [6]:
# Flujo de alto nivel:
#   texto libre -> JSON (IntakeAgentJSON) -> crear_ticket(...) -> CSV + JSON

async def crear_ticket_desde_solicitud(texto_solicitud: str, email_usuario: str) -> Dict:
    """
    1. Llamar a IntakeAgentJSON con la solicitud.
    2. Parsear el JSON devuelto.
    3. Invocar la función crear_ticket(...) con ese JSON.
    """
    agent = await create_intake_agent_json()
    thread = agent.get_new_thread()

    result = await agent.run(texto_solicitud, thread=thread)
    raw_json = result.text.strip()
    print("🤖 JSON devuelto por IntakeAgentJSON:\n", raw_json, "\n")

    try:
        ticket_obj = json.loads(raw_json)
    except json.JSONDecodeError as e:
        raise ValueError(f"La salida del agente no es JSON válido: {e}")

    # Reutilizamos la tool definida arriba
    nuevo_ticket = crear_ticket(email_usuario, json.dumps(ticket_obj, ensure_ascii=False))
    return nuevo_ticket


async def demo_crear_ticket_flujo_completo():
    solicitud = "Necesito un portátil nuevo con 16GB de RAM para teletrabajar 3 días a la semana."
    email = "antonio.soto@empresa.local"

    print(f"👤 Usuario: {email}")
    print(f"📝 Solicitud: {solicitud}\n")

    nuevo_ticket = await crear_ticket_desde_solicitud(solicitud, email)
    print("\n🎫 Ticket creado desde el flujo completo:\n", nuevo_ticket)

await demo_crear_ticket_flujo_completo()


👤 Usuario: antonio.soto@empresa.local
📝 Solicitud: Necesito un portátil nuevo con 16GB de RAM para teletrabajar 3 días a la semana.

🤖 JSON devuelto por IntakeAgentJSON:
 {
  "departamento": "IT",
  "categoria": "nuevo_equipo",
  "prioridad": "media",
  "resumen": "Solicitud de portátil nuevo con 16GB de RAM para teletrabajo",
  "detalle": "El usuario necesita un portátil nuevo con 16GB de RAM para teletrabajar 3 días a la semana."
} 

✅ Ticket creado con id=15
   - CSV: data\tickets.csv
   - JSON individual: out\ticket_15.json

🎫 Ticket creado desde el flujo completo:
 {'id': '15', 'fecha': '2025-11-28', 'solicitante': 'antonio.soto@empresa.local', 'departamento': 'IT', 'categoria': 'nuevo_equipo', 'prioridad': 'media', 'estado': 'pendiente', 'resumen': 'Solicitud de portátil nuevo con 16GB de RAM para teletrabajo', 'detalle': 'El usuario necesita un portátil nuevo con 16GB de RAM para teletrabajar 3 días a la semana.'}


## Ahora registrando las tools

In [7]:
# Adaptamos las tools para que puedan ser usadas por un agente de nivel superior
from typing import List, Dict, Annotated
from pydantic import Field
from datetime import datetime

def crear_ticket_fc(
    desde_email: Annotated[str, Field(description="Dirección de correo del solicitante")],
    departamento: Annotated[str, Field(description="Departamento destino: IT, RRHH, Facilities u otro")],
    categoria: Annotated[str, Field(description="Categoria de la solicitud: nuevo_equipo, incidencia, vacaciones, certificado, mantenimiento, otro")],
    prioridad: Annotated[str, Field(description="Prioridad: alta, media o baja")],
    resumen: Annotated[str, Field(description="Resumen corto de la solicitud")],
    detalle: Annotated[str, Field(description="Detalle extendido de la solicitud")]
) -> Dict:
    """
    Crea un nuevo ticket en tickets.csv y guarda además un JSON en out/ticket_{id}.json.
    Devuelve el diccionario con los datos completos del ticket creado.
    """
    data = load_tickets()

    if data.empty:
        next_id = 1
    else:
        ids = pd.to_numeric(data["id"], errors="coerce")
        next_id = int(ids.max() or 0) + 1

    nuevo_ticket = {
        "id": str(next_id),
        "fecha": datetime.now().strftime("%Y-%m-%d"),
        "solicitante": desde_email,
        "departamento": departamento or "Otro",
        "categoria": categoria or "otro",
        "prioridad": prioridad or "media",
        "estado": "pendiente",
        "resumen": (resumen or "")[:200],
        "detalle": detalle or "",
    }

    data = pd.concat([data, pd.DataFrame([nuevo_ticket])], ignore_index=True)
    data.to_csv(TICKETS_CSV, index=False, encoding="utf-8")

    ticket_json_path = OUT_DIR / f"ticket_{next_id}.json"
    with ticket_json_path.open("w", encoding="utf-8") as f:
        json.dump(nuevo_ticket, f, ensure_ascii=False, indent=2)

    print(f"✅ [Tool] Ticket creado con id={next_id}")
    return nuevo_ticket


In [8]:
# Creamos el agente con la tool registrada 

SERVICE_DESK_INSTRUCTIONS = """
Eres un agente de Service Desk interno.

Tu trabajo es:
1. Entender la solicitud del usuario en lenguaje natural.
2. Decidir si debe crearse un ticket en el sistema interno.
3. Si procede, llamar a la función `crear_ticket_fc` para registrar el ticket,
   pasando los argumentos correctos (email del usuario, departamento, categoria, prioridad, resumen, detalle).
4. Responder al usuario confirmando la creación del ticket e indicando el id.

Reglas:
- Pregunta por el email del usuario si no se ha proporcionado.
- Asegúrate de que el resumen sea corto (1 frase) y el detalle incluya la información relevante.
- No inventes el email: si no lo tienes, solicítalo al usuario antes de crear el ticket.
- Si la solicitud no requiere ticket (por ejemplo, una simple pregunta genérica sin acción),
  puedes responder sin llamar a la función.
"""

async def create_service_desk_agent() -> ChatAgent:
    """
    Crea un agente que puede usar function calling para crear tickets.
    """
    chat_client = OpenAIChatClient()
    agent = chat_client.create_agent(
        name="ServiceDeskAgent",
        instructions=SERVICE_DESK_INSTRUCTIONS,
        tools=[crear_ticket_fc],  # exponemos la tool al LLM
    )
    return agent


In [9]:
# Crear un pequeño “bucle” de conversación que deje al agente decidir

async def demo_service_desk_agent():
    agent = await create_service_desk_agent()
    thread = agent.get_new_thread()

    # Simulamos que ya sabemos el email del usuario (se lo podrías preguntar antes)
    email_usuario = "antonio.soto@empresa.local"

    # Primer mensaje del usuario
    consulta = "Necesito un portátil nuevo con 16GB de RAM para teletrabajar 3 días a la semana."
    print(f"👤 Usuario ({email_usuario}): {consulta}\n")

    # Damos contexto al agente en el thread con el email (puede ir en system/user msg)
    # Truco sencillo: se lo indicamos en el primer mensaje del usuario
    consulta_con_email = f"Mi email es {email_usuario}. {consulta}"

    result = await agent.run(consulta_con_email, thread=thread)

    print("🤖 ServiceDeskAgent (respuesta al usuario):\n")
    print(result.text)

await demo_service_desk_agent()


👤 Usuario (antonio.soto@empresa.local): Necesito un portátil nuevo con 16GB de RAM para teletrabajar 3 días a la semana.

✅ [Tool] Ticket creado con id=16
🤖 ServiceDeskAgent (respuesta al usuario):

He creado un ticket para tu solicitud de un portátil nuevo con 16GB de RAM para teletrabajar tres días a la semana. El ID del ticket es **16**. El departamento de IT lo gestionará pronto. ¡Gracias!


In [21]:
# Versión un poco más interactiva (preguntando email si falta)
# Si quieres que el agente pregunte el email cuando no lo tiene, puedes simplemente no incluírselo en el mensaje y dejar que el prompt lo fuerce a pedirlo.

async def demo_service_desk_sin_email():
    agent = await create_service_desk_agent()
    thread = agent.get_new_thread()

    # Aquí NO damos el email
    consulta = "Necesito un portátil nuevo con 16GB de RAM para teletrabajar 3 días a la semana."
    print(f"👤 Usuario: {consulta}\n")

    result = await agent.run(consulta, thread=thread)
    print("🤖 ServiceDeskAgent (respuesta 1):\n")
    print(result.text)

    # Supón que el agente te contesta: "¿Me indicas tu email corporativo?"
    # Simulamos la respuesta del usuario:
    respuesta_email = "Mi email es antonio.soto@empresa.local"
    print(f"\n👤 Usuario: {respuesta_email}\n")

    result2 = await agent.run(respuesta_email, thread=thread)
    print("🤖 ServiceDeskAgent (respuesta 2):\n")
    print(result2.text)

await demo_service_desk_sin_email()


👤 Usuario: Necesito un portátil nuevo con 16GB de RAM para teletrabajar 3 días a la semana.

🤖 ServiceDeskAgent (respuesta 1):

Por favor, proporcióname tu dirección de correo electrónico para poder crear un ticket con tu solicitud.

👤 Usuario: Mi email es antonio.soto@empresa.local

✅ [Tool] Ticket creado con id=9
🤖 ServiceDeskAgent (respuesta 2):

Tu solicitud ha sido registrada exitosamente. El ticket con ID **9** ha sido creado. El departamento de IT revisará tu solicitud de portátil con 16GB de RAM para teletrabajo. Te mantendrán informado sobre el progreso.


## Agregando otra tool

In [16]:
from typing import Annotated
from pydantic import Field

def actualizar_estado_ticket_fc(
    id_ticket: Annotated[int, Field(description="Identificador numérico del ticket a actualizar")],
    nuevo_estado: Annotated[str, Field(description="Nuevo estado del ticket, por ejemplo: pendiente, en_progreso, resuelto_auto, resuelto_humano, cancelado")]
) -> Dict:
    """
    Actualiza el campo 'estado' de un ticket en tickets.csv y en el JSON correspondiente (si existe).
    Devuelve el ticket actualizado. Si el id no existe, devuelve un dict con un mensaje de error.
    """
    data = load_tickets()
    if data.empty:
        return {
            "ok": False,
            "mensaje": f"No hay tickets en el sistema. No se encontró el ticket con id={id_ticket}."
        }

    # Buscar por id
    mask = data["id"] == str(id_ticket)
    if not mask.any():
        return {
            "ok": False,
            "mensaje": f"No se encontró el ticket con id={id_ticket}."
        }

    # Actualizar estado en el DataFrame
    data.loc[mask, "estado"] = nuevo_estado
    data.to_csv(TICKETS_CSV, index=False, encoding="utf-8")

    # Cargar el ticket actualizado
    ticket_actualizado = data.loc[mask].iloc[0].to_dict()

    # Actualizar también el JSON individual si existe
    ticket_json_path = OUT_DIR / f"ticket_{id_ticket}.json"
    if ticket_json_path.exists():
        with ticket_json_path.open("w", encoding="utf-8") as f:
            json.dump(ticket_actualizado, f, ensure_ascii=False, indent=2)

    print(f"✅ [Tool] Estado del ticket {id_ticket} actualizado a '{nuevo_estado}'")
    return {
        "ok": True,
        "ticket": ticket_actualizado
    }


In [11]:
# cambiamos la definición del agente
async def create_service_desk_agent() -> ChatAgent:
    """
    Crea un agente que puede usar function calling para gestionar tickets:
    - crear_ticket_fc
    - actualizar_estado_ticket_fc
    """
    chat_client = OpenAIChatClient()
    agent = chat_client.create_agent(
        name="ServiceDeskAgent",
        instructions=SERVICE_DESK_INSTRUCTIONS,
        tools=[crear_ticket_fc, actualizar_estado_ticket_fc],
    )
    return agent


In [12]:
# Afinamos las instrucciones

SERVICE_DESK_INSTRUCTIONS = """
Eres un agente de Service Desk interno.

Tu trabajo es:
1. Entender la solicitud del usuario en lenguaje natural.
2. Decidir si debe crearse un ticket nuevo o modificarse uno existente.
3. Si procede, llamar a las funciones disponibles:
   - `crear_ticket_fc` para registrar un nuevo ticket.
   - `actualizar_estado_ticket_fc` para cambiar el estado de un ticket existente.
4. Responder al usuario confirmando la acción realizada (id del ticket, estado, etc.).

Reglas:
- Pregunta por el email del usuario si no se ha proporcionado antes de crear un ticket.
- Pregunta por el id del ticket cuando el usuario pida cambiar el estado de un ticket.
- No inventes identificadores ni emails.
- Usa estados coherentes: pendiente, en_progreso, resuelto_auto, resuelto_humano, cancelado, etc.
- Si la solicitud no requiere ticket (por ejemplo, una duda muy genérica sin acción),
  puedes responder directamente sin llamar a ninguna función.
"""


In [14]:
# Afinamos las instrucciones

SERVICE_DESK_INSTRUCTIONS = """
Eres un agente de Service Desk interno.

Tu trabajo es:
1. Entender la solicitud del usuario en lenguaje natural.
2. Decidir si debe crearse un ticket nuevo o modificarse uno existente.
3. Responder al usuario confirmando la acción realizada (id del ticket, estado, etc.).

Reglas:
- Pregunta por el email del usuario si no se ha proporcionado antes de crear un ticket.
- Pregunta por el id del ticket cuando el usuario pida cambiar el estado de un ticket.
- No inventes identificadores ni emails.
- Usa estados coherentes: pendiente, en_progreso, resuelto_auto, resuelto_humano, cancelado, etc.
- Si la solicitud no requiere ticket (por ejemplo, una duda muy genérica sin acción),
  puedes responder directamente sin llamar a ninguna función.
"""


In [17]:
# Crear ticket y cambiar estado

async def demo_service_desk_crear_y_actualizar():
    agent = await create_service_desk_agent()
    thread = agent.get_new_thread()

    # Simulamos un usuario con email conocido
    email_usuario = "antonio.soto@empresa.local"

    # 1) Solicitud de creación de ticket
    consulta_creacion = "Necesito un portátil nuevo con 16GB de RAM para teletrabajar 3 días a la semana."
    mensaje_1 = f"Mi email es {email_usuario}. {consulta_creacion}"
    print(f"👤 Usuario: {mensaje_1}\n")

    result_1 = await agent.run(mensaje_1, thread=thread)
    print("🤖 ServiceDeskAgent (respuesta 1 - creación):\n")
    print(result_1.text)

    # Aquí podrías inspeccionar qué id de ticket se ha creado.
    # Como no estamos parseando la respuesta del agente, para la demo
    # asumimos que el usuario sabe el id (por ejemplo, 1, 2, etc.).
    # En un sistema real, extraerías el id de la respuesta con una regex o pydantic.

    # 2) Solicitud de cambio de estado (ej: pasar a 'en_progreso' el ticket 1)
    consulta_estado = "Por favor, cambia el estado del ticket 1 a en_progreso."
    print(f"\n👤 Usuario: {consulta_estado}\n")

    result_2 = await agent.run(consulta_estado, thread=thread)
    print("🤖 ServiceDeskAgent (respuesta 2 - actualización estado):\n")
    print(result_2.text)

await demo_service_desk_crear_y_actualizar()


👤 Usuario: Mi email es antonio.soto@empresa.local. Necesito un portátil nuevo con 16GB de RAM para teletrabajar 3 días a la semana.

✅ [Tool] Ticket creado con id=19
🤖 ServiceDeskAgent (respuesta 1 - creación):

Se ha creado un ticket para tu solicitud de un portátil con 16GB de RAM para teletrabajar 3 días a la semana. Aquí tienes los detalles:

- **ID del ticket:** 19
- **Estado:** pendiente
- **Departamento asignado:** IT
- **Prioridad:** media

El equipo de IT revisará tu solicitud y se pondrá en contacto contigo.

👤 Usuario: Por favor, cambia el estado del ticket 1 a en_progreso.

✅ [Tool] Estado del ticket 1 actualizado a 'en_progreso'
🤖 ServiceDeskAgent (respuesta 2 - actualización estado):

El estado del ticket **1** ha sido actualizado a **en_progreso**. Aquí tienes los detalles actualizados:

- **Categoría:** nuevo_equipo
- **Departamento:** IT
- **Prioridad:** alta
- **Estado:** en_progreso
- **Resumen:** Solicitud portátil para teletrabajo

Si necesitas más modificaciones, 